In [1]:
import os, sys
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "intelliwiz_config,settings.py")
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
import django
django.setup()

In [5]:
# Model imports
from apps.activity.models import Asset,AssetLog, Attachment, Device, DeviceEventlog, Event, Job, Jobneed, JobneedDetails, Location, Question, QuestionSet, QuestionSetBelonging, WorkPermit
from apps.attendance.models import PeopleEventlog, TestGeo, Tracking
from apps.onboarding.models import Bt,  GeofenceMaster, Shift, TypeAssist
from apps.peoples.models import Capability, People, PermissionGroup, Pgbelonging, Pgroup
from apps.reminder.models import Reminder
from apps.tenants.models import Tenant
from apps.y_helpdesk.models import EscalationMatrix, Ticket
from apps.core import utils
import pandas as pd

# Shell Plus Django Imports
from django.db import transaction
from django.db.models import Avg, Case, Count, F,Func, Max, Min, Prefetch, Q, Sum, When, ExpressionWrapper, Value as V
from django.db.models.functions import Concat, Cast
from django.utils import timezone
from django.urls import reverse
from django.db.models import Exists, OuterRef, Subquery, FloatField, CharField, IntegerField
from datetime import datetime, timedelta

In [3]:
import pandas as pd
from IPython.display import display, clear_output
import ipywidgets as widgets
from IPython.display import display as ipy_display

def dq(queryset):
    if not queryset: return None
    if queryset := list(queryset):
    
        if isinstance(queryset, (list, tuple)):
            # Handle values_list() queryset
            df = pd.DataFrame(queryset)
        elif isinstance(queryset, dict):
            # Handle values() queryset
            df = pd.DataFrame.from_records([queryset])
        else:
            # Handle regular queryset
            data = list(queryset.values())
            df = pd.DataFrame(data)

        # Set pandas display options to show all rows and columns
        pd.set_option('display.max_rows', None)
        pd.set_option('display.max_columns', None)
        pd.set_option('display.width', None)

        # Get the specified column names
        columns = list(queryset[0].keys())

        # Filter the DataFrame based on the specified columns
        df_filtered = df[columns]

        # Create a scrollable output widget
        output_widget = widgets.Output()

        # Display the DataFrame inside the output widget
        with output_widget:
            clear_output()
            display(df_filtered)

        # Display the output widget
        ipy_display(output_widget)
    else:
        print("Empty Queryset")

In [4]:
datetime.utcnow()

datetime.datetime(2023, 8, 29, 11, 48, 14, 832480)

In [7]:
lat_lng_array = list(
    Location.objects.annotate(
        lat=Func('gpslocation', function='ST_X'),
        lng=Func('gpslocation', function='ST_Y'),
    ).values('lat', 'lng')
)

# Convert the queryset to the desired format
checkpoints = [[float(item['lat']), float(item['lng'])] for item in lat_lng_array]

# Output the checkpoints array
print(f"checkpoints = {checkpoints}")

ProgrammingError: function st_x(geography) does not exist
LINE 1: SELECT ST_X("location"."gpslocation")::bytea AS "lat", ST_Y(...
               ^
HINT:  No function matches the given name and argument types. You might need to add explicit type casts.
